[TOC]

# Type System

类型系统是 Julia 的精华

## 类型系统

### 类型的分类

- abstract type，定义行为
- concrete type，定义数据的存储方式
  - 原始类型
  - 复合类型

### 类型申明

`::`类型申明，有两大作用：

#### 检查表达式输出结果

1. 写在表达式最后
2. 不能检查赋值语句中`=`左边的全局变量（函数内没问题）

In [ ]:
s = 'a'::String # 注意，String 检查的是 = 后面整个表达式的类型

报错，表明输出不是预想的 String

In [ ]:
s = 'a'::Char

In [ ]:
b = Int64(3.0::Float64) # 检查 3.0

In [ ]:
b = convert(Int, 3.0)::Int64 # 检查 convert(Int, 3.0)

#### 提升程序性能

使用中，对类型的规范越细致，程序运行速度越快

In [ ]:
function foo()
  x::Int8 = 1.0
  x
end

In [ ]:
foo() # 自动将类型转换为了 Int8

### 查看类型

`typeof()`

`eltype()` 数组中元素的类型

`typemin()`/`typemax()`查看类型能够表达的最小/大值

`supertype()`/`supertypes()`/`subtypes()`查看类型之间的关系

`subtypetree()` 自定义函数，已在 startup.jl 中加载，查看树状子类型关系图

In [22]:
# Display the entire type hierarchy starting from the specified `roottype`
function subtypetree(roottype, level=1, indent=4)
    level == 1 && println(roottype)
    for s in subtypes(roottype)
        println(join(fill(" ", level * indent)) * string(s))
        subtypetree(s, level + 1, indent)
    end
end

subtypetree (generic function with 3 methods)

### `is..()` 判断类型

返回 `Bool`

`isodd()`/`iseven()` 判断奇偶数

`isinteger()` 判断是否为整数

`isfinite()`/`isinf()` 判断是否为无穷值

`isnan(x)` `x` 是否是 `NaN`

`isdigit(char)` 判断一个字符是否在 `'0':'9'` 的范围内

### Type Conversion

#### `convert(T, x)`/ 构造函数`T()`

**`convert()` 性能更好**

构造函数包括 `Int64()`, `Float64()`, `UInt8()`, `BigInt()`, etc

In [1]:
convert(Float64, 1) 

1.0

In [5]:
Float64(3)

3.0

#### `parse(T, string; base)`

将一个 base 进制的字符串解析为 10 进制数字

#### `trunc([T,] x)`

`trunc([T,] x)` 截取整数部分（不论小数部分多大）
`trunc(x; digits::Integer= [, base = 10])`
`trunc(x; sigdigits::Integer= [, base = 10])`

#### `string()`

转换为字符串

#### `big()`

将数据类型升格为更大的类型，Int 变 BigInt，Float 变 BigFloat

#### 自动转换

1. 赋值给数组会将值转换为该数组的元素类型。
2. 赋值给对象的字段会将值转换为该字段的声明类型。
3. 使用new构造对象会将值转换为该对象的声明的字段类型。
4. 赋值给具有声明类型的变量会将值转换为该类型。
5. 具有声明的返回类型的函数会将其返回值转换为该类型。
6. 将值传递给ccall会将值转换为相应的参数类型。

In [6]:
x = rand(3)

3-element Vector{Float64}:
 0.9247724162546608
 0.2431208477977339
 0.6736535587819961

In [8]:
x[1] = 1
x

3-element Vector{Float64}:
 1.0
 0.2431208477977339
 0.6736535587819961

In [13]:
mutable struct Foo
    x::Float64
end

foo = Foo(1) # 使用 new 构造对象

Foo(1.0)

In [12]:
foo.x = 2
foo

Foo(2.0)

In [15]:
function foo2()
    local x::Float64
    x = 1
    println(x, " has type of ", typeof(x))
end
foo2()

1.0 has type of Float64


In [18]:
function foo3()::Float64
    return 1
end
foo3()

1.0

但是，**传递参数不会自动转换类型**

In [20]:
twice = x::AbstractFloat -> 2x
twice(1.2)

2.4

In [21]:
twice(1) # 会报错

LoadError: MethodError: no method matching (::var"#1#2")(::Int64)
[0mClosest candidates are:
[0m  (::var"#1#2")([91m::AbstractFloat[39m) at In[20]:1

### Type Unions

比如一个函数的输入可以是整数或字符串，如何进行类型申明？就要构造 type union

`Union{Types...}` 任何对象都不是它的实例

In [ ]:
IntOrString = Union{Int64, String}

In [ ]:
input_value = Union{Missing, Int64}

### Type Promotion

In [ ]:
promote(1, 2.5, 1//2)

### 类型与性能

尽量保持变量类型的稳定，比如一个整数如果参与除法，就尽量在定义这个变量时将其声明为浮点型

In [ ]:
x = 1.0

## Abstract Type


- 抽象类无法被实例化
  - 比如正多边形是正三角形和正方形的父类。后两者是具体类，有相应的数据储存格式，而前者只是一个用来说明后两者有某些共同行为的抽象类，在内存中没有具体实现，仅作为一种标记行为的工具。

  - 在处理抽象类型时，可以只关注特定的行为以及它们之间可能的交互。

  - 这就是传说中的鸭子类型，不管它里面放着什么数据，只要行为像鸭子，它就是鸭子。即根据行为而非计算机实现来标注类型。

- 具体类之间不能互为子类型（必然是类型层次结构中的叶子节点），只有抽象类可以作为其他类型的超类

- 抽象类一般用*斜体*表示

`Any` 是顶级类型，所有类都是它的子类




## Built-in Types

也被称为原始类型，相对于复合类型

### Number

#### Integer

##### 整数取值范围的上限

| 类型                                                         | 带符号？ | 比特数 | 最小值    | 最大值    |
| :----------------------------------------------------------- | :------- | :----- | :-------- | :-------- |
| [Int8](https://docs.julialang.org/en/v1/base/numbers/#Core.Int8) | ✓        | 8      | -2^7      | 2^7 – 1   |
| [UInt8](https://docs.julialang.org/en/v1/base/numbers/#Core.UInt8) |          | 8      | 0         | 2^8 – 1   |
| [Int16](https://docs.julialang.org/en/v1/base/numbers/#Core.Int16) | ✓        | 16     | -2^15     | 2^15 – 1  |
| [UInt16](https://docs.julialang.org/en/v1/base/numbers/#Core.UInt16) |          | 16     | 0         | 2^16 – 1  |
| [Int32](https://docs.julialang.org/en/v1/base/numbers/#Core.Int32) | ✓        | 32     | -2^31     | 2^31 – 1  |
| [UInt32](https://docs.julialang.org/en/v1/base/numbers/#Core.UInt32) |          | 32     | 0         | 2^32 – 1  |
| [Int64](https://docs.julialang.org/en/v1/base/numbers/#Core.Int64) | ✓        | 64     | -2^63     | 2^63 – 1  |
| [UInt64](https://docs.julialang.org/en/v1/base/numbers/#Core.UInt64) |          | 64     | 0         | 2^64 – 1  |
| [Int128](https://docs.julialang.org/en/v1/base/numbers/#Core.Int128) | ✓        | 128    | -2^127    | 2^127 – 1 |
| [UInt128](https://docs.julialang.org/en/v1/base/numbers/#Core.UInt128) |          | 128    | 0         | 2^128 – 1 |
| [Bool](https://docs.julialang.org/en/v1/base/numbers/#Core.Bool) | N/A      | 8      | false (0) | true (1)  |

整数类型值域的上限不足，是 Julia 一个非常突出的特点。这是为了速度优化而对数据进行严格分类的一种牺牲。

注意，integer 类型的取值范围是比较小的，因此如果程序中某个表达式需要处理的数值大小超过 $2^{64}$ 乃至 $2^{128}$，就会溢出。必须将数据转换为 Float 再计算。

转换的方法很灵活，运算过程中任何一个数是浮点数，结果都会是浮点数。

In [ ]:
typemax(Int64) # 2^63 - 1
# 9223372036854775807

In [ ]:
typemax(Float64)
# Inf

In [ ]:
1000^21
# -9223372036854775808

In [ ]:
Float64(1000)^21
# 1.0e63

In [ ]:
(1000+0.0)^21
# 1.0e63

上述计算结果（大于 $2^{64}$ 乃至 $2^{128}$ 的浮点数）仍然可以被转换为整数，但必须用 `BigInt` 格式来储存。

转换函数为 `big()` 或 `BigInt()`，或使用 `convert(BigInt, x)`

In [ ]:
x = BigInt(10212313131352519345963644753026192783913791739137917391739137)

In [ ]:
convert(BigInt, Float64(1000)^21)

##### 进位制

前缀区分进制：

\- `0x` 十六进制

\- `0b` 二进制

\- `0o` 八进制


10 进制（数字）与其他进制（字符串）的互转

- `parse(type, str; base)` 将 base 进制的字符串解析为 10 进制整数
- `string(n; base, pad)` 将 10 进制整数转换为 base 进制的字符串，并补齐至 pad 长度
- 忽略 base 时，则是 10 进制数字与字符串的互转

In [ ]:
parse(Int64, "FF"; base=16)

In [ ]:
string(12; base=16, pad=2)

#### Floating-point

用科学计数法输入数字时，默认为浮点数（可能因为整数的值域太小了）

In [ ]:
2e2

在计算机能够表达的浮点数集合中，越靠近零点，数值的分布越稠密；而远离零点时，则会变得越来越稀疏，精度也会越来越差。

##### NaN

not a number, 用 `isnan()` 检查是否 NaN

In [ ]:
0/0 # NaN

In [ ]:
typeof(NaN) # Float64

##### 无穷大

Julia 允许除数为 0，返回 Inf 或 -Inf，用 `isfinite()`, `isinf()` 检查是否无穷大

> 但不允许分子分母同时为0

In [ ]:
isfinite(Inf) # false

In [ ]:
isinf(-Inf) # true

#### 复数

In [ ]:
1+im  |> typeof # Complex{Int64}

#### 有理数

`Rational{T<:Integer>}<:Real` 分子、分母必须都是整数

`numerator()`, `denominator()` 分别返回标准化的分子和分母

In [ ]:
1 // 2 # 分数

In [ ]:
typeof(1 // 2)

In [ ]:
numerator(1//2)

In [ ]:
denominator(1//2)

数学上等价的有理数，在 Julia 中表达形式唯一

- 分子与分母为不同整型时，Julia会通过必要的隐式转换，将两者的类型进行统一
- 创建的Rational数值在Julia内部会被约分为标准形式
- 确保分母不为负数
- 不允许分子、分母同时为 0
- 在 julia 底层，比较两个分数值是否相等时，是通过**校验分子与分母都相等**来实现的

In [ ]:
UInt32(2)//Int64(10) # 统一为 Int64

In [ ]:
typeof(UInt32(2) // Int64(10))

In [ ]:
5//25

In [ ]:
1//-2

In [ ]:
5//0

In [ ]:
0//0

In [ ]:
2//3 == 6//9

In [ ]:
5//8 * 3//12

In [ ]:
6//5 / 10//7

#### 无理数

julia 有一些内置常数，如 ℯ (**\euler TAB**)

In [ ]:
typeof(ℯ)

### Nothing

没有返回值的函数，返回 `nothing`，其 type 为 `Nothing`

### Missing

In [ ]:
typeof(missing) # Missing

在包含 missing 的数组上调用归约函数会返回 missing，需要先用 `skipmissing()` 跳过缺失值

In [ ]:
a = [1, missing]

In [ ]:
typeof(a)

In [ ]:
sum(a)

In [ ]:
skipmissing(a)

In [ ]:
skipmissing(a) |> sum

### Bool

true/false 参与数值计算会自动转化为 1/0

In [ ]:
true == 1

In [ ]:
false == 0

In [ ]:
true == 2

In [ ]:
Int64(true) # 1

In [ ]:
Int64(false) # 0

In [ ]:
Bool(1::Int64) # true

In [ ]:
Bool(0) # false

In [ ]:
Bool(10) # 非 1 和 0 的数值转换为 Bool 型会报错！这是 Julia 的一个特点

### Symbol

以`:`开始，可以和 String 相互转换

In [ ]:
sym = :some_text

In [ ]:
typeof(sym)

In [ ]:
s = string(sym)

In [ ]:
sym = Symbol(s)

但 Symbol 不能像 String 一样使用数字索引

### Char

#### 构造

字符的类型为 `Char`，父类型为 `AbstractChar`，形式为一对单引号 `'x'`，且其中只能有一个字符

In [ ]:
typeof('x')

In [ ]:
'x'

In [ ]:
# 除了提示该字符的 Unicode 码值（16进制）外，还告知其在字符集中的分类
'￥'

也可以用 `Char(数字)` 创建字符

In [ ]:
Char(0x78)

In [ ]:
Char(120) # 10进制的120，对应16进制的78

In [ ]:
Int('x')

#### 运算

字符之间的四则运算仅支持减法，返回二者编码之间的距离

In [ ]:
'x' - 'a'

In [ ]:
'x' - 23

In [ ]:
'A' + 1

#### 判断分类

`isletter(c::AbstractChar)` 是否在 letter 类中

`isascii(c::Union(AbstractChar, AbstractString))` Char 是否在 ASCII 中，或 String 的所有字符是否都在 ASCII 中

`isnumber(c::AbstractChar)` 是否在 numeric 类中

`isdigit(c::AbstractChar)` 是否为 0-9 的数字

`isprint(c::AbstractChar)` 是否为可打印字符

`iscntrl(c::AbstractChar)` 是否为不可打印字符（如换行符、制表符）

`ispunct(c::AbstractChar)` 是否为标点符号

`isspace(c::AbstractChar)` 是否为白空格

`islowercase(c::AbstractChar)` 是否为小写字符

`isuppercase(c::AbstractChar)` 是否为大写字符

## 复合类型

不可变对象的字段如果是可变的（如数组），这个字段就仍然能被改写。

可变对象的所有字段都能被改写。

不可变对象更易于处理，但空间开销更大。

### 参数化复合类型

复合类型接收一些参数，这些参数对字段的类型进行了某种规定，并在实例化时具体确定。

In [23]:
# 抽象类型
abstract type Asset end
abstract type Investment <: Asset end
abstract type Equity <: Investment end

# 具体类型
struct Stock <: Equity
    symbol::String
    name::String
end

# 参数化类型
struct StockHolding{T <: Real} 
    stock::Stock
    quantity::T
end

实例化时，有可能出现 StockHolding{Int}，也有可能出现 StockHolding{Float}

In [24]:
stock = Stock("AAPL", "Apple, Inc.")

Stock("AAPL", "Apple, Inc.")

In [25]:
holding = StockHolding(stock, 100)

StockHolding{Int64}(Stock("AAPL", "Apple, Inc."), 100)

In [26]:
holding = StockHolding(stock, 100.00)

StockHolding{Float64}(Stock("AAPL", "Apple, Inc."), 100.0)

In [27]:
holding = StockHolding(stock, 100 // 3)

StockHolding{Rational{Int64}}(Stock("AAPL", "Apple, Inc."), 100//3)

参数化类型可以强制约定字段类型的一致性

In [28]:
struct StockHolding2{T<:Real,P<:AbstractFloat}
    stock::Stock
    quantity::T
    price::P
    marketvalue::P
end

In [29]:
holding = StockHolding2(stock, 100, 180.00, 18000.00)

StockHolding2{Int64, Float64}(Stock("AAPL", "Apple, Inc."), 100, 180.0, 18000.0)

In [30]:
abstract type Holding{P} end

mutable struct StockHolding3{T,P} <: Holding{P}
    stock::Stock
    quantity::T
    price::P
    marketvalue::P
end

mutable struct CashHolding{P} <: Holding{P}
    currency::String
    amount::P
    marketvalue::P
end

In [31]:
certificate_in_the_safe = StockHolding3(stock, 100, 180.00, 18000.00)

StockHolding3{Int64, Float64}(Stock("AAPL", "Apple, Inc."), 100, 180.0, 18000.0)

In [32]:
certificate_in_the_safe isa Holding{Float64}

true